In [ ]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import re
import base64

import animal_shelter as CRUD

# Data Model
username = "aacuser"
password = "aacpass"
animals = CRUD.AnimalShelter(password, username)

# Set up documents from database
df = pd.DataFrame.from_records(animals.read({}))



# Dashboard Layout
app = JupyterDash('MyPetProject')
image_filename = 'Grazioso.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Add Grozioso Details
app.layout = html.Div([
    html.A([
        html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
                             height = 250, width = 251))], href = 'https://www.snhu.edu', target = "_blank"),
    html.Center(html.B(html.H1('Isa\' SNHU CS-340 Dashboard'))),
    html.Hr(),
    html.Div(
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain Rescue', 'value': 'mountain'},
                {'label': 'Disaster Rescue', 'value': 'disaster'},
                {'label': 'All Animals', 'value': 'reset'}
            ],
            value='reset'
        )

    ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=True,
        row_selectable="single",
        selected_rows=[],
        filter_action="native",
        sort_action="native",
        page_action="native",
        page_current=0,
        page_size=10,

    ),
    html.Br(),
    html.Hr(),

    html.Div(className='row',
             style={'display' : 'flex', 'justify-content':'center'},
             children=[
                 html.Div(
                     id='graph-id',
                     className='col s12 m6',
                 ),
                 html.Div(
                     id='map-id',
                     className='col s12 m6',
                 )
             ])
])

# Components
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(bt1, bt2, bt3, bt4):
    ### FIX ME Add code to filter interactive data table with MongoDB queries
    if (int(bt1) > 0):
        df = pd.DataFrame.from_records(shelter.read({'$and': [
            {'$or': [ {'breed':'Labrador Retriever Mix'}, {'breed':'Chesapeake Bay Retriever'},
                      {'breed':'Newfoundland'}]},
            {'sex_upon_outcome':'Intact Female'}, {'age_upon_outcome_in_weeks':{'$lte':26, '$gte':156}}]}))
        bt2 = 0
        bt3 = 0
        bt4 = 0
    elif (int(bt2) > 0):
        df = pd.DataFrame.from_records(shelter.read({'$and': [
            {'$or': [ {'breed':'German Shepherd'}, {'breed':'Alaskan Malamute'},
                      {'breed':'Old English Sheepdog'},{'breed':'Siberian Husky'},{'breed':'Rottweiler'}]},
            {'sex_upon_outcome':'Intact Male'}, {'age_upon_outcome_in_weeks':{'$lte':26, '$gte':156}}]}))
        bt1 = 0
        bt3 = 0
        bt4 = 0
    elif (int(bt3) > 0):
        df = pd.DataFrame.from_records(shelter.read({'$and': [
            {'$or': [ {'breed':'Doberman Pinscher'}, {'breed':'German Sheperd'},
                      {'breed':'Golden Retriever'},{'breed':'Bloodhound'},{'breed':'Rottweiler'}]},
            {'sex_upon_outcome':'Intact Male'}, {'age_upon_outcome_in_weeks':{'$lte':20, '$gte':300}}]}))
        bt1 = 0
        bt2 = 0
        bt4 = 0
    elif (int(bt4) > 0):
        df = pd.DataFrame.from_records(shelter.read({}))
        bt1 = 0
        bt2 = 0
        bt3 = 0
        bt4 = 0

    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')

    return (data,columns)

# change colors
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#808080'
    } for i in selected_columns]

# Pie Chart
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(data):
    # add code for chart of your choice (e.g. pie chart) #
    df = pd.DataFrame.from_dict(data)

    fig = px.pie(
        df,
        names={'breed'},
    )

    return fig


@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX location marker
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[30.75,-97.48], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1,9])
                ])
            ])
        ])
    ]

app